In [1]:
import os
from dotenv import load_dotenv

load_dotenv('token.env')
api_token = os.getenv('api_token')


In [2]:
# from langchain.document_loaders import PyPDFLoader

# loader = PyPDFLoader("Accelerating-Apache-Spark-3.pdf")
# data = loader.load()

from langchain.document_loaders import TextLoader

loader = TextLoader("instructions.txt")
data = loader.load()

In [3]:
data[0].page_content

'[General Gameplay/Start Game]\n- Provide occasional hints, tips, or prompts to guide the player\'s decision-making process.\n- Encourage creativity and exploration while maintaining a sense of challenge and adventure.\n\n[Exploration]\n- The player finds themselves in a dense, enchanted forest teeming with mystical creatures and hidden treasures.\n- Encourage the player to explore the forest by describing the surroundings, potential encounters, and points of interest.\n- Use vivid imagery and descriptive language to immerse the player in the forest environment.\n- Provide hints or clues about nearby landmarks, valuable items, or potential dangers.\n\n[What to do]\n- Exploration: The player can move deeper into the forest to discover new areas and encounters.\n- Combat: Engaging in combat with nearby monsters is an option.\n- Rest: The player can take a break to regain health and stamina.\n- Treasure Hunting: They can search for chests containing valuable loot.\n\n[Monster Encounter/Co

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter

# splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1000,
#     chunk_overlap=100
# )

# data = splitter.split_documents(data)
# len(data)

# r_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=450,
#     chunk_overlap=0, 
#     separators=["\n\n"]
# )

from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=10, chunk_overlap=1)

In [5]:
# text_splitter = RecursiveCharacterTextSplitter(separators=["forest"], chunk_size=450, chunk_overlap=0)
docs = text_splitter.split_documents(data)

Created a chunk of size 214, which is longer than the specified 10
Created a chunk of size 430, which is longer than the specified 10
Created a chunk of size 311, which is longer than the specified 10
Created a chunk of size 814, which is longer than the specified 10
Created a chunk of size 328, which is longer than the specified 10


In [6]:
dash_line = "-" * 100
print(len(docs))
for doc in docs:
    print(doc.page_content)
    print(dash_line + '\n' + dash_line)


6
[General Gameplay/Start Game]
- Provide occasional hints, tips, or prompts to guide the player's decision-making process.
- Encourage creativity and exploration while maintaining a sense of challenge and adventure.
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
[Exploration]
- The player finds themselves in a dense, enchanted forest teeming with mystical creatures and hidden treasures.
- Encourage the player to explore the forest by describing the surroundings, potential encounters, and points of interest.
- Use vivid imagery and descriptive language to immerse the player in the forest environment.
- Provide hints or clues about nearby landmarks, valuable items, or potential dangers.
----------------------------------------------------------------------------------------------------
-------------------------------------------------

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = 'sentence-transformers/all-MiniLM-L6-v2'
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [8]:
from langchain.vectorstores import Chroma

In [9]:
vectordb = Chroma.from_documents(
    documents=docs,
    embedding = embeddings,
    persist_directory = 'instructions'
    )

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "google/gemma-2b-it"
dtype = torch.bfloat16

tokenizer = AutoTokenizer.from_pretrained(model_name,token = api_token)
model = AutoModelForCausalLM.from_pretrained(model_name,token = api_token, device_map = 'auto', torch_dtype = dtype)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
from transformers import pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA

pipe = pipeline(
 "text-generation", 
 model=model, 
 tokenizer=tokenizer,
 return_tensors='pt',
 max_new_tokens=200,
 model_kwargs={"torch_dtype": torch.bfloat16}
)

In [28]:
llm = HuggingFacePipeline(
 pipeline=pipe,
 model_kwargs={"temperature": 0.5},
)

In [29]:
qa = RetrievalQA.from_chain_type(
 llm=llm,
 chain_type="stuff",
 retriever=vectordb.as_retriever(search_kwargs={"k":4}),

)

In [36]:
start_prompt = '''GAME DIALOGUE:
model:
The beast lunges at you, its claws outstretched. You narrowly avoid its attack and find yourself in a standoff, your strength against its fury. You know that you must find a way to defeat this beast, but you are running out of time. What will you do next?<end_of_turn>

based on the dialogue above, give relevant instructions to model to continue the story.'''

In [37]:
qa.invoke(start_prompt)

{'query': 'GAME DIALOGUE:\nmodel:\nThe beast lunges at you, its claws outstretched. You narrowly avoid its attack and find yourself in a standoff, your strength against its fury. You know that you must find a way to defeat this beast, but you are running out of time. What will you do next?<end_of_turn>\n\nbased on the dialogue above, give relevant instructions to model to continue the story.',
 'result': "\nThe player must quickly find a way to defend themselves against the beast's attack. They can use any available weapon or tactic to defeat the beast."}